In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical

import random

In [ ]:
WIDTH=300
HEIGHT=300
IMAGE_SIZE=(WIDTH, HEIGHT)
IMAGE_CHANNELS=3
batch_size = 12

In [ ]:
pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile

with ZipFile('../content/drive/MyDrive/GlassMetal.zip', 'r') as zip:
    zip.extractall()


# with py7zr.SevenZipFile('../content/drive/MyDrive/GlassMetal.zip', mode='r') as z:
#     z.extractall()

path = '../content/GlassMetal'

filenames = os.listdir(path)

In [ ]:
#delete dirs
shutil.rmtree( train_dir )
shutil.rmtree( validation_dir )



In [ ]:
path =  "GlassMetal"
train_dir = os.path.join(path, "train")
validation_dir = os.path.join(path, "validation")

train_1_dir = os.path.join(train_dir, 'glass')
train_2_dir = os.path.join(train_dir, 'metal')

validation_1_dir = os.path.join(validation_dir, 'glass')
validation_2_dir = os.path.join(validation_dir,  'metal')


os.makedirs(train_dir)
os.makedirs(validation_dir)
os.makedirs(train_1_dir)
os.makedirs(train_2_dir)
os.makedirs(validation_1_dir)
os.makedirs(validation_2_dir)

In [ ]:
df = pd.DataFrame(data=filenames, columns=['file_name'])
total_train, total_validate = 0, 0

df['label'] = df['file_name'].apply(lambda x: 'glass' if x[:3]=='gla'
                                    else "metal")

for label in df['label'].unique():
    a = label
    files = df[df['label']==label]['file_name']
    train = np.random.choice(files, size=int(len(files)*0.8))
    test = [_ for _ in files if not (_ in train)]
    print(label, len(files), len(train), len(test))
    total_train += len(train)
    total_validate += len(test)

    for file in train:
      shutil.copyfile(os.path.join(path, file),
                     os.path.join(train_dir, a, file))


    for file in test:
      shutil.copyfile(os.path.join(path, file),
                     os.path.join(validation_dir, a, file))


glass 501 400 230
metal 410 328 177


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 149, 149, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 73, 73, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 35, 35, 64)      

In [ ]:

from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator( rescale = 1.0/255. ,
                                   rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )


In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=IMAGE_SIZE )

validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = IMAGE_SIZE)

Found 504 images belonging to 2 classes.
Found 407 images belonging to 2 classes.


In [ ]:
history = model.fit(
            train_generator,
            epochs=15,
            validation_data=validation_generator,
            verbose=2
            )

Epoch 1/15
26/26 - 16s - loss: 0.8543 - accuracy: 0.4960 - val_loss: 0.6837 - val_accuracy: 0.5676 - 16s/epoch - 605ms/step
Epoch 2/15
26/26 - 13s - loss: 0.6947 - accuracy: 0.5139 - val_loss: 0.6871 - val_accuracy: 0.5651 - 13s/epoch - 494ms/step
Epoch 3/15
26/26 - 12s - loss: 0.6901 - accuracy: 0.5456 - val_loss: 0.7003 - val_accuracy: 0.4349 - 12s/epoch - 475ms/step
Epoch 4/15
26/26 - 11s - loss: 0.6955 - accuracy: 0.5417 - val_loss: 0.6597 - val_accuracy: 0.5872 - 11s/epoch - 425ms/step
Epoch 5/15
26/26 - 13s - loss: 0.7003 - accuracy: 0.6111 - val_loss: 0.6232 - val_accuracy: 0.7224 - 13s/epoch - 489ms/step
Epoch 6/15
26/26 - 13s - loss: 0.6339 - accuracy: 0.6587 - val_loss: 0.5550 - val_accuracy: 0.7273 - 13s/epoch - 517ms/step
Epoch 7/15
26/26 - 13s - loss: 0.6132 - accuracy: 0.6885 - val_loss: 0.9569 - val_accuracy: 0.5332 - 13s/epoch - 496ms/step
Epoch 8/15
26/26 - 13s - loss: 0.5747 - accuracy: 0.7083 - val_loss: 0.7238 - val_accuracy: 0.6143 - 13s/epoch - 505ms/step
Epoch 9/